In [1]:
# en este notebook generaremos un archivo parquet para la función user data y luego definimos la función para probarla antes de
# pasarla al main.py
import pandas as pd 


In [3]:
df_user = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\users_items.parquet', columns=['user_id', 'item_id', 'items_count'])
df_steam = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\steam_games.parquet', columns= ['id', 'price'])

df = df_user.merge(df_steam, how='left', left_on= 'item_id', right_on= 'id')

del df_user
del df_steam

df.drop(columns=['item_id', 'id'], inplace=True)
df = df.groupby('user_id').agg({'items_count': 'max', 'price': 'sum'}).reset_index()
df.rename(columns={'price': 'dinero_gastado'}, inplace=True)
df_reviews = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\reviews.parquet', columns= ['user_id', 'recommend'])
df_reviews = df_reviews.groupby('user_id').agg('sum').reset_index()

df = df.merge(df_reviews, how= 'left')
del df_reviews

df['porcentaje_recom'] = round(df['recommend']*100 / df['items_count'], 2)
df.drop(columns='recommend', inplace=True)
df.fillna(0, inplace=True)
df.head()

df.to_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\userdata.parquet')
del df


In [4]:
def userdata(user_id: str):

    if not isinstance(user_id, str):
        return {'Mensaje': 'El argumento user_id debe ser una cadena de texto.'}

    df = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\userdata.parquet')
    df= df[df['user_id'].isin([user_id])].reset_index(drop=True)
    
    if df.empty:
        del df
        return {'Mensaje': 'Usuario no encontrado. Por favor ingrese un usuario válido'}
    
    df = df.reset_index(drop=True)

    resultado = {
        'Usuario': user_id,
        'Dinero gastado': str(round(df.loc[0,'dinero_gastado'], 2)) + ' USD',
        '% de recomendación': str(df.loc[0,'porcentaje_recom']) + ' %',
        'Cantidad de Items': int(df.loc[0,'items_count'])
    }

    return resultado

In [5]:
userdata('jonasdbomb')

{'Usuario': 'jonasdbomb',
 'Dinero gastado': '5352.1 USD',
 '% de recomendación': '0.22 %',
 'Cantidad de Items': 461}